In [3]:
!pip install xgboost

     ---------------------------------------- 99.7/99.7 MB 3.2 MB/s eta 0:00:00


In [4]:
#import packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

In [6]:
df = pd.read_csv('hr_train.csv')
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.42,0.46,2,150,3,0,1,0,sales,medium
1,0.66,0.77,2,171,2,0,0,0,technical,medium
2,0.55,0.49,5,240,3,0,0,0,technical,high
3,0.22,0.88,4,213,3,1,0,0,technical,medium
4,0.20,0.72,6,224,4,0,1,0,technical,medium


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10499 entries, 0 to 10498
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     10499 non-null  float64
 1   last_evaluation        10499 non-null  float64
 2   number_project         10499 non-null  int64  
 3   average_montly_hours   10499 non-null  int64  
 4   time_spend_company     10499 non-null  int64  
 5   Work_accident          10499 non-null  int64  
 6   left                   10499 non-null  int64  
 7   promotion_last_5years  10499 non-null  int64  
 8   sales                  10499 non-null  object 
 9   salary                 10499 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 820.4+ KB


In [33]:
df.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000,10499.000000
mean,0.612683,0.717131,3.808553,201.059815,3.494238,0.144299,0.292885,0.021716
std,0.248578,0.171483,1.230572,49.959332,1.453227,0.351410,0.455108,0.145763
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [9]:
df.left.value_counts()

0    7424
1    3075
Name: left, dtype: int64

In [31]:
print(f"There are", (round(df[df['left']==1].left.sum()/len(df.left)*100,2)), "% of employees left the company.")

There are 29.29 % of employees left the company.


In [41]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [42]:
xgb_params = {  
                "learning_rate":[0.01,0.05,0.1,0.3,0.5],
                "gamma":[i/10.0 for i in range(0,5)],
                "max_depth": [2,3,4,5,6,7,8],
                "min_child_weight":[1,2,5,10],
                "max_delta_step":[0,1,2,5,10],
                "subsample":[i/10.0 for i in range(5,10)],
                "colsample_bytree":[i/10.0 for i in range(5,10)],
                "colsample_bylevel":[i/10.0 for i in range(5,10)],
                "reg_lambda":[1e-5, 1e-2, 0.1, 1, 100], 
                "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
                "scale_pos_weight":[1,2,3,4,5,6,7,8,9],
                "n_estimators":[100,500,700,1000]
             }

In [40]:
xgb=XGBClassifier(objective='binary:logistic')

In [44]:
n_iter=10

random_search=RandomizedSearchCV(xgb,n_jobs=-1,cv=5,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=xgb_params)

In [49]:
X_train = df.drop(['left','sales','salary'],axis=1)
y_train = df['left']

In [50]:
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.01

In [51]:
report(random_search.cv_results_,3)

Model with rank: 1
Mean validation score: 0.838 (std: 0.01981)
Parameters: {'subsample': 0.9, 'scale_pos_weight': 4, 'reg_lambda': 1e-05, 'reg_alpha': 1, 'n_estimators': 100, 'min_child_weight': 2, 'max_depth': 2, 'max_delta_step': 1, 'learning_rate': 0.1, 'gamma': 0.0, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.9}

Model with rank: 2
Mean validation score: 0.837 (std: 0.01898)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 2, 'reg_lambda': 1e-05, 'reg_alpha': 100, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 8, 'max_delta_step': 1, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.9}

Model with rank: 3
Mean validation score: 0.836 (std: 0.01917)
Parameters: {'subsample': 0.5, 'scale_pos_weight': 4, 'reg_lambda': 0.01, 'reg_alpha': 100, 'n_estimators': 500, 'min_child_weight': 2, 'max_depth': 8, 'max_delta_step': 1, 'learning_rate': 0.5, 'gamma': 0.1, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.9}



In [52]:
xgb5=XGBClassifier(learning_rate=0.1,n_estimators=100,min_child_weight=2,gamma=12,max_depth=8,
                   scale_pos_weight=1,max_delta_step=0,
                   colsample_bylevel= 1.0, colsample_bytree= 0.8, subsample= 0.7
                  )

In [53]:
xgb_params={
            'reg_lambda':[i/10 for i in range(0,50)],
            'reg_alpha':[i/10 for i in range(0,50)]
            }

In [56]:
random_search=RandomizedSearchCV(xgb5,param_distributions=xgb_params,cv=5,n_iter=20,scoring='roc_auc',
                                n_jobs=-1,verbose=10)

In [57]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=1.0,
                                           colsample_bynode=None,
                                           colsample_bytree=0.8, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=12, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=0.1...
                                           random_state=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_dist

In [58]:
report(random_search.cv_results_, 3)

Model with rank: 1
Mean validation score: 0.837 (std: 0.02089)
Parameters: {'reg_lambda': 3.8, 'reg_alpha': 0.3}

Model with rank: 2
Mean validation score: 0.837 (std: 0.02116)
Parameters: {'reg_lambda': 2.6, 'reg_alpha': 0.6}

Model with rank: 3
Mean validation score: 0.837 (std: 0.02092)
Parameters: {'reg_lambda': 1.4, 'reg_alpha': 0.8}



In [59]:
xgb6=XGBClassifier(learning_rate=0.1,n_estimators=100,min_child_weight=2,gamma=12,max_depth=8,
                   scale_pos_weight=1,max_delta_step=0,
                   colsample_bylevel= 1.0, colsample_bytree= 0.8, subsample= 0.7,
                  reg_lambda=1.8,reg_alpha=0)

In [61]:
xgb6.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=1.0, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=12, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=0,
              max_depth=8, max_leaves=None, min_child_weight=2, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [63]:
from sklearn.model_selection import cross_val_score

cross_val_score(xgb6,X_train,y_train,scoring='roc_auc',verbose=10,n_jobs=-1,cv=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


array([0.83733635, 0.84409458, 0.80154549, 0.8086144 , 0.85702471,
       0.88402287, 0.83630935, 0.83472015, 0.83279994, 0.83578145])